# Jupyter notebook to calculate the timeseries of Amplitude and Phase used in the GRL paper: "Heat extremes driven by amplification of phase-locked circumglobal waves forced by topography in an idealized atmospheric model" by B. Jimenez-Esteve, K. Kornhuber and D. I.V. Domeisen

Author: Bernat Jimenez-Esteve (ETH Zurich) Last update: October 2022

The notebook blocks need to be executed sequencially.

The netCDF files containing the daily model output can be must be downloaded from the following repository: 


In [14]:
#load main modeules
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#choose lat band
lat_band = slice(60,30)

### Model data

In [3]:
#Read ICON data
f = 'data/daily/u300_v300_z300_t1000_heldsuarez_notopo_daymean.nc'
v_hs =  xr.open_dataset(f).v300.sel(lat=lat_band,time=slice('0001-01-01', '0030-12-30')).mean('lat').load()

## 1. Apply a 7-days running mean (weekly values)

In [9]:
#apply a 7-day running mean to get rid of synoptic scale eddies
v_7d = v_hs.rolling(time=7, center=True).mean().dropna('time')

## 2. Fast-Fourier Transform (FFT)

In [10]:
from scipy.fft import fft, rfft

#Fast fourier transform
ffty = rfft(v_7d.values,axis=-1)

#amplitude and phase calculation for different wave numbers
N=len(v_7d[0,:]) #number of longitude points
amp_cal = 2.0/N * np.abs(ffty[:,0:N//2])
phase_cal = (-1) * np.arctan2(np.imag(ffty[:,0:N//2]),np.real(ffty[:,0:N//2]))

In [11]:
#we create an x_array with coordinates (time,lat,wavenumber) 
k =np.arange(0,11,1) #we only keep the first 10 wavenumbers

#this array contains the amplitude for different wavenumbers at different times and latitudes
amplitudes = xr.DataArray(coords=[v_7d.time, k],
                           dims=['time', 'wavenumber'],
                          data= amp_cal[:,0:11] ) 

phases = xr.DataArray(coords=[v_7d.time, k],
                           dims=['time', 'wavenumber'],
                          data= phase_cal[:,0:11] ) 

## 3. Save file

In [38]:
#save this in a file
dir="/data/test.nc"
ds =  xr.Dataset(coords=amplitudes.coords)

#add the new data
ds = ds.assign(amplitudes  = amplitudes)
ds = ds.assign(phases  = phases)
ds.to_netcdf(dir)